# Phase 1: Data Discovery & Analysis
### Data Engineering Assignment

#### 👤 Author: Sarvesh Bajare
#### 🏢 Role: Data Engineering Intern
#### 📅 Date: 21/06/25

---

## 🧠 Objective
This notebook initiates the first phase of building a unified data pipeline by:
- Exploring three disparate datasets from recently acquired e-commerce platforms.
- Identifying data quality issues and inconsistencies.
- Mapping relationships across datasets to form a normalized schema.
- Strategizing the complete data cleaning approach.

---

## 📁 Datasets Overview

| Source | File Name | Format | Contents |
|--------|-----------|--------|----------|
| Platform A | `customers_messy_data.json` | JSON | Customer records |
| Platform B | `products_inconsistent_data.json` | JSON | Product catalog |
| Platform C | `orders_unstructured_data.csv` | CSV | Order transaction logs |


#### Step 1: 
First I will start with loading the data from multiple sources. For now, I am doing this by simply loading it from srotage with native csv and json handling techniques. In further versions this can be done by simulating data retreival from multi cloud based data storages. 

In [27]:
import pandas as pd
import json

# csv
orders_df = pd.read_csv("datasets/orders_unstructured_data.csv")

# json file 1
with open("datasets/customers_messy_data.json") as f:
    customer_json = json.load(f)
# Fix: If customer_json is a list of dicts (records), load directly
if isinstance(customer_json, list):
    customers_df = pd.DataFrame(customer_json)
else:
    customers_df = pd.DataFrame([customer_json]) 

# json file 2
with open("datasets/products_inconsistent_data.json") as f:
    product_json = json.load(f)
# Fix: If product_json is a list of dicts (records), load directly
if isinstance(product_json, list):
    products_df = pd.DataFrame(product_json)
else:
    products_df = pd.DataFrame([product_json])


#### Step 2:
Inspecting the Datasets for their shape (size) and inclusion of various datatypes. Here, both json files will have objects as contains (still checking to avoid any anamolies during the transform phase).

In [28]:
# preview each dataset
display(orders_df.head())
display(customers_df.head())
display(products_df.head())

# Shape check
print("Orders shape:", orders_df.shape)
print("Customers shape:", customers_df.shape)
print("Products shape:", products_df.shape)

# Data types
print("Orders dtypes:")
print(orders_df.dtypes)

print("\nCustomers dtypes:")
print(customers_df.dtypes)

print("\nProducts dtypes:")
print(products_df.dtypes)


,order_id,ord_id,customer_id,cust_id,order_date,order_datetime,product_id,item_id,quantity,qty,...,order_total,shipping_cost,tax,discount,status,order_status,payment_method,shipping_address,notes,tracking_number
0,ORD_00001,1,109,CUST_0109,NaN,2023-10-09T02:35:00.000Z,PROD_165,63,6,4,...,710.25,17.20,6.83,48.89,returned,SHIPPED,cash,608 Oak Ave,NaN,TRK769590
1,ORD_00002,2,279,CUST_0279,1/26/2023,NaN,PROD_154,186,7,1,...,63.82,16.11,17.13,10.09,NaN,CANCELLED,bank_transfer,5636 Main St,NaN,TRK965629
2,ORD_00003,3,6,CUST_0006,NaN,2023-12-12T14:59:00.000Z,PROD_070,115,1,4,...,656.66,24.84,7.46,0.00,CANCELLED,CANCELLED,debit_card,7410 Oak Ave,NaN,TRK280164
3,ORD_00004,4,248,CUST_0248,7/17/2023,2023-10-13T16:44:00.000Z,PROD_102,182,9,3,...,118.88,14.14,41.12,0.00,SHIPPED,NaN,credit_card,4492 Oak Ave,NaN,NaN
4,ORD_00005,5,353,CUST_0353,8/23/2023,2023-09-09T04:55:00.000Z,PROD_197,145,4,1,...,246.92,6.45,14.05,0.00,returned,SHIPPED,paypal,8501 Pine Rd,NaN,TRK710064


,customer_id,cust_id,customer_name,full_name,email,email_address,phone,phone_number,address,city,...,status,customer_status,total_orders,total_spent,loyalty_points,preferred_payment,age,birth_date,gender,segment
0,1,CUST_0001,Grace Lee,Charlie Brown,customer1@example.com,user1@domain.com,None,,3108 Oak Ave,NYC,...,suspended,None,22,1830.44,272,paypal,NaN,None,Female,None
1,2,CUST_0002,Bob Wilson,Bob Wilson,None,user2@domain.com,555-5457,,9256 Second Ave,Los Angeles,...,None,ACTIVE,25,2107.53,37,None,51.0,1971-07-27,,None
2,3,CUST_0003,Grace Lee,Bob Wilson,customer3@example.com,user3@domain.com,None,(555) 170-7757,3136 First St,Chicago,...,active,None,15,351.82,225,None,41.0,None,None,new
3,4,CUST_0004,John Smith,diana.prince,None,user4@domain.com,555-3017,(555) 266-3492,9294 Main St,new_york,...,INACTIVE,INACTIVE,27,155.54,576,credit_card,NaN,None,F,vip
4,5,CUST_0005,henry.davis123,Charlie Brown,customer5@example.com,user5@domain.com,None,,5012 Oak Ave,Phoenix,...,suspended,pending,5,3762.96,132,credit_card,59.0,1998-11-18,None,None


,product_id,item_id,product_name,item_name,description,category,product_category,brand,manufacturer,price,...,color,size,stock_quantity,stock_level,reorder_level,supplier_id,created_date,last_updated,is_active,rating
0,PROD_001,1,Product 1,Item 1 Name,Description for product 1,clothing,clothing,BRAND-C,BrandA,102.18,...,,,538,282,13,None,None,2023-08-10T18:30:00.000Z,no,2.7
1,PROD_002,2,Product 2,Item 2 Name,Description for product 2,Home & Garden,Electronics,BrandA,brand_b,259.94,...,Black,L,859,33,29,SUP_04,2022-11-08,2023-07-19T18:30:00.000Z,1,4.0
2,PROD_003,3,Product 3,Item 3 Name,Description for product 3,CLOTHING,Toys,brand_b,Brand D,200.05,...,None,S,815,251,36,SUP_05,2020-09-25,2023-01-06T18:30:00.000Z,False,None
3,PROD_004,4,Product 4,Item 4 Name,Description for product 4,Sports,electronics,brandE,brandE,254.98,...,Black,M,278,429,39,SUP_15,2020-11-21,2023-04-04T18:30:00.000Z,0,3.1
4,PROD_005,5,Product 5,Item 5 Name,Description for product 5,Electronics,Electronics,Brand D,Brand D,237.76,...,Red,,463,249,18,SUP_16,None,2023-04-27T18:30:00.000Z,False,None


Orders shape: (1000, 23)
Customers shape: (500, 25)
Products shape: (200, 24)
Orders dtypes:
order_id             object
ord_id                int64
customer_id           int64
cust_id              object
order_date           object
order_datetime       object
product_id           object
item_id               int64
quantity              int64
qty                   int64
unit_price          float64
price               float64
total_amount        float64
order_total         float64
shipping_cost       float64
tax                 float64
discount            float64
status               object
order_status         object
payment_method       object
shipping_address     object
notes                object
tracking_number      object
dtype: object

Customers dtypes:
customer_id           object
cust_id               object
customer_name         object
full_name             object
email                 object
email_address         object
phone                 object
phone_number          objec

#### Step 3:
Profiling of all three datasets. Getting crucial null counts and column overlaps. There is a visual nulls' map at the end to understand the measure and compare null count for each column visually.

In [29]:
# 3.1: Orders Table Profiling

# counting nulls
orders_nulls = orders_df.isnull().sum().to_frame("null_count")
orders_nulls["null_%"] = (orders_nulls["null_count"] / len(orders_df)) * 100
display(orders_nulls.sort_values("null_%", ascending=False))

# looking for overlaps
overlapping_cols = ["order_id", "ord_id", "customer_id", "cust_id", 
                    "product_id", "item_id", "quantity", "qty", 
                    "unit_price", "price", "total_amount", "order_total"]

for col in overlapping_cols:
    if col in orders_df.columns:
        print(f"{col}: {orders_df[col].nunique()} unique values")


,null_count,null_%
notes,798,79.8
tracking_number,423,42.3
order_datetime,201,20.1
status,160,16.0
order_status,126,12.6
order_date,117,11.7
order_id,0,0.0
product_id,0,0.0
cust_id,0,0.0
ord_id,0,0.0


order_id: 1000 unique values
ord_id: 1000 unique values
customer_id: 431 unique values
cust_id: 431 unique values
product_id: 196 unique values
item_id: 198 unique values
quantity: 10 unique values
qty: 5 unique values
unit_price: 995 unique values
price: 988 unique values
total_amount: 993 unique values
order_total: 990 unique values


In [30]:
 # 3.2: Customers Table Profiling

# counting nulls
customers_nulls = customers_df.isnull().sum().to_frame("null_count")
customers_nulls["null_%"] = (customers_nulls["null_count"] / len(customers_df)) * 100
display(customers_nulls.sort_values("null_%", ascending=False))


# looking for overlaps
overlapping_customer_cols = ["customer_id", "cust_id", 
                             "customer_name", "full_name",
                             "email", "email_address",
                             "phone", "phone_number",
                             "registration_date", "reg_date",
                             "status", "customer_status"]

for col in overlapping_customer_cols:
    if col in customers_df.columns:
        print(f"{col}: {customers_df[col].unique()}")


,null_count,null_%
phone,301,60.2
birth_date,295,59.0
age,226,45.2
zip_code,150,30.0
preferred_payment,106,21.2
segment,105,21.0
email,98,19.6
registration_date,98,19.6
gender,70,14.0
customer_status,54,10.8


customer_id: ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '

In [31]:
# 3.3: Products Table Profiling

# counting nulls
products_nulls = products_df.isnull().sum().to_frame("null_count")
products_nulls["null_%"] = (products_nulls["null_count"] / len(products_df)) * 100
display(products_nulls.sort_values("null_%", ascending=False))

# looking for overlaps
overlapping_product_cols = ["product_id", "item_id", 
                            "product_name", "item_name",
                            "category", "product_category",
                            "brand", "manufacturer",
                            "price", "list_price", "cost",
                            "stock_quantity", "stock_level"]

for col in overlapping_product_cols:
    if col in products_df.columns:
        print(f"{col}: {products_df[col].unique()}")


,null_count,null_%
weight,74,37.0
rating,68,34.0
supplier_id,61,30.5
description,57,28.5
created_date,35,17.5
size,33,16.5
manufacturer,31,15.5
brand,30,15.0
color,21,10.5
category,0,0.0


product_id: ['PROD_001' 'PROD_002' 'PROD_003' 'PROD_004' 'PROD_005' 'PROD_006'
 'PROD_007' 'PROD_008' 'PROD_009' 'PROD_010' 'PROD_011' 'PROD_012'
 'PROD_013' 'PROD_014' 'PROD_015' 'PROD_016' 'PROD_017' 'PROD_018'
 'PROD_019' 'PROD_020' 'PROD_021' 'PROD_022' 'PROD_023' 'PROD_024'
 'PROD_025' 'PROD_026' 'PROD_027' 'PROD_028' 'PROD_029' 'PROD_030'
 'PROD_031' 'PROD_032' 'PROD_033' 'PROD_034' 'PROD_035' 'PROD_036'
 'PROD_037' 'PROD_038' 'PROD_039' 'PROD_040' 'PROD_041' 'PROD_042'
 'PROD_043' 'PROD_044' 'PROD_045' 'PROD_046' 'PROD_047' 'PROD_048'
 'PROD_049' 'PROD_050' 'PROD_051' 'PROD_052' 'PROD_053' 'PROD_054'
 'PROD_055' 'PROD_056' 'PROD_057' 'PROD_058' 'PROD_059' 'PROD_060'
 'PROD_061' 'PROD_062' 'PROD_063' 'PROD_064' 'PROD_065' 'PROD_066'
 'PROD_067' 'PROD_068' 'PROD_069' 'PROD_070' 'PROD_071' 'PROD_072'
 'PROD_073' 'PROD_074' 'PROD_075' 'PROD_076' 'PROD_077' 'PROD_078'
 'PROD_079' 'PROD_080' 'PROD_081' 'PROD_082' 'PROD_083' 'PROD_084'
 'PROD_085' 'PROD_086' 'PROD_087' 'PROD_088' 'PROD

In [ ]:
# Visual null map  
import missingno as msno
msno.matrix(orders_df)
msno.matrix(customers_df)
msno.matrix(products_df)


## 🔍 Data Quality Summary (Draft)

Each dataset exhibits redundancy and inconsistency across multiple columns. Below are the initial findings:

### Orders Dataset
- Duplicate IDs: `order_id`, `ord_id`
- Product & customer ID fields duplicated
- Quantity & price split across aliases
- Mixed null representations and stringified numerics

### Customers Dataset
- Conflicting names, emails, registration dates
- Multiple null representations in phone and age fields
- `status` varies in casing and completeness

### Products Dataset
- Field duplication (`product_name`, `item_name`; `brand`, `manufacturer`)
- Prices and costs as strings
- `is_active` field as a string instead of boolean
- Conflicts in inventory fields (`stock_quantity`, `stock_level`)

These will be consolidated in the cleaning phase.


#### Step 4:

Relationship Mapping

In [35]:
# 4.1 Identify Primary Keys

# Orders
print("Unique order_id:", orders_df['order_id'].nunique())
print("Total rows:", len(orders_df))

# Customers
print("Unique customer_id:", customers_df['customer_id'].nunique())

# Products
print("Unique product_id:", products_df['product_id'].nunique())


Unique order_id: 1000
Total rows: 1000
Unique customer_id: 500
Unique product_id: 200


In [34]:
# 4.2 Discover Foreign Key Relationships

# Check if customer_id in orders exists in customers
valid_customer_ids = set(customers_df['customer_id'])
invalid_customers = orders_df[~orders_df['customer_id'].isin(valid_customer_ids)]

print("Invalid customer references in orders:", len(invalid_customers))

# Same for product_id
valid_product_ids = set(products_df['product_id'])
invalid_products = orders_df[~orders_df['product_id'].isin(valid_product_ids)]

print("Invalid product references in orders:", len(invalid_products))


Invalid customer references in orders: 1000
Invalid product references in orders: 0


Mismatch in Foreign Keys Invalid customer references in orders: 1000 This means None of the customer_id values in orders match the customer_id values in customers.

In [ ]:
# testing a different mapping for cust_id

valid_cust_ids = set(customers_df['cust_id'])
invalid_customers_v2 = orders_df[~orders_df['cust_id'].isin(valid_cust_ids)]

print("Invalid CUST_IDs from orders:", len(invalid_customers_v2))


Invalid CUST_IDs from orders: 0


## ✅ Final Entity-Relationship Model (ERD)

After analysis and testing, the following relationships were discovered and confirmed:

### 🎯 Primary Keys:
- `orders`: `order_id`
- `customers`: `cust_id` (chosen over `customer_id`)
- `products`: `product_id`

### 🔗 Foreign Key Relationships:
- `orders.cust_id → customers.cust_id`
- `orders.product_id → products.product_id`

---

## 🧼 Redundant or Misleading Columns:
- `order_id` vs `ord_id` → use `order_id`
- `customer_id` vs `cust_id` → use `cust_id`
- `item_id` vs `product_id` → use `product_id`

These decisions guide the cleaning strategy and normalized schema design for Phase 2.


#### Step 6: 
Data Cleaning Strategy Planning

## 🧹 Data Cleaning Strategy

Based on the profiling and relationship mapping, the following data cleaning plan is proposed:

### Orders Cleaning
- Keep: `order_id`, `cust_id`, `product_id`, `quantity`, `unit_price`, `order_total`, `tax`, `discount`, `shipping_cost`, `payment_method`, `shipping_address`, `order_datetime`
- Drop: redundant aliases (`ord_id`, `qty`, `price`, `total_amount`)
- Merge `status` and `order_status` → `order_status_clean`
- Convert numeric and datetime fields

### Customers Cleaning
- Keep: `cust_id`, `full_name`, `email_address`, `phone_number`, `address`, `city`, `state`, `zip_code`, `registration_date`, `status`, `total_spent`, `loyalty_points`, `preferred_payment`, `gender`
- Drop: `customer_id`, `customer_name`, duplicate columns
- Standardize: status casing, null values, phone number formatting
- Convert types (e.g., `total_spent → float`)

### Products Cleaning
- Keep: `product_id`, `product_name`, `description`, `category`, `brand`, `price`, `cost`, `stock_quantity`, `reorder_level`, `is_active`, `rating`
- Drop or rename: redundant fields (`item_name`, `manufacturer`)
- Convert: all pricing and ratings to floats
- Map: `is_active` → Boolean

These transformations will be implemented in modular cleaning functions in Phase 2.
